In [2]:
import pandas as pd
import time
from datetime import datetime
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By # 셀레니움 4.0부터 포함된 객체(모듈)
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.keys import Keys
import warnings
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_note_interactivity="all"

In [2]:
def setting():
    # 드라이버 설정
    global driver
    chrome_options = webdriver.ChromeOptions() # 크롬브라우저를 컨트롤하는 드라이버 옵션 생성
    chrome_options.add_experimental_option("detach", True)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

    # url 로딩
    url = "https://tossinvest.com/stocks/A005930/community"
    driver.get(url)
    time.sleep(5)

In [ ]:
stock_name = driver.find_element(By.XPATH, '//*[@id="__next"]/div[1]/div[1]/main/div/div/div/div[3]/div/div[3]/div[1]/span[1]')
stock_name.text

In [4]:
# 버튼을 클릭해서 인기순 정렬 최신순으로 변경해야함
def sorting():
    sort_button = driver.find_element(By.CSS_SELECTOR, 'section._1m9brd0 > button')
    sort_std = sort_button.get_attribute('data-contents-label')
    if sort_std == "인기순":
        sort_button.click()
    if sort_std == "최신순":
        print('현재 정렬이 최신순입니다.')

In [61]:
# 댓글, 시간 수집
collected_comments = []

def comment_collect(repeat_count):
    global collected_comments  # 리스트에 계속 저장되도록 글로벌 변수 사용

    for _ in range(repeat_count):
        # 현재 화면에서 보이는 댓글 요소 찾기
        comment_elements = driver.find_elements(By.CSS_SELECTOR, "[data-index]")

        for idx, comment in enumerate(comment_elements, start=1):  # nth-child 순서대로 수집
            try:
                # 댓글 내용 가져오기
                text_selector = f"#stock-content > div > div > section > section > ul > div > div:nth-child({idx}) > article > div > a > span:nth-child(2) > span"
                text_element = driver.find_element(By.CSS_SELECTOR, text_selector)
                comment_text = text_element.text.strip()

                # 작성 시간 가져오기
                time_selector = f"#stock-content > div > div > section > section > ul > div > div:nth-child({idx}) > article > div > header > div > label > time"
                time_element = driver.find_element(By.CSS_SELECTOR, time_selector)
                timestamp = time_element.get_attribute('datetime')

                # 중복 방지
                if (comment_text, timestamp) not in collected_comments:
                    collected_comments.append((comment_text, timestamp))

            except Exception as e:
                print(f"오류 발생: {e}")
                continue

        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN)
        time.sleep(3)  # 페이지 로딩 대기

comment_collect(5)

오류 발생: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#stock-content > div > div > section > section > ul > div > div:nth-child(1) > article > div > a > span:nth-child(2) > span"}
  (Session info: chrome=134.0.6998.89); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0089B5A3+24387]
	(No symbol) [0x00825904]
	(No symbol) [0x00700753]
	(No symbol) [0x00748BA9]
	(No symbol) [0x00748EFB]
	(No symbol) [0x007919C2]
	(No symbol) [0x0076D894]
	(No symbol) [0x0078F138]
	(No symbol) [0x0076D646]
	(No symbol) [0x0073C59F]
	(No symbol) [0x0073D8E4]
	GetHandleVerifier [0x00B9D883+3179043]
	GetHandleVerifier [0x00BB6CF9+3282585]
	GetHandleVerifier [0x00BB167C+3260444]
	GetHandleVerifier [0x00934330+650448]
	(No symbol) [0x0082ED0D]
	(No symbol) [0x0082BAF8]
	(No symbol) [0x0082BC99]
	(No symbol) [0x0081E530]
	BaseThreadInitThunk

In [24]:
# 결과를 Pandas DataFrame으로 변환
def save_comments(filename="test_comments.csv"):
    global collected_comments

    try:
        df_existing = pd.read_csv(filename, encoding="utf-8-sig")
    except FileNotFoundError:
        df_existing = pd.DataFrame(columns=["comment", "timestamp"])

    # 새로 수집한 데이터를 DataFrame으로 변환
    df_new = pd.DataFrame(collected_comments, columns=["comment", "timestamp"])

    # 기존 데이터와 합치기 (중복 제거)
    df_combined = pd.concat([df_existing, df_new]).drop_duplicates().reset_index(drop=True)

    # CSV로 저장 (덮어쓰기)
    df_combined.to_csv(filename, index=False, encoding="utf-8-sig")

    # ✅ 저장 후 리스트 초기화 (이미 저장한 데이터는 다시 중복되지 않도록)
    collected_comments.clear()

    save_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"✅ {filename} 파일로 저장 완료! 현재 총 {len(df_combined)}개의 댓글이 저장됨. 마지막 저장 시간: {save_time}")



In [27]:
driver.quit()

In [ ]:
# # 변수선언부
# collected_ids = set()  # 중복 방지용 ID 저장
# collected_comments = []  # 수집된 댓글 저장 리스트

# # id확인
# def load_existing_ids(file_path="test_comments.csv"):
#     global collected_ids
#     try:
#         existing_df = pd.read_csv(file_path, encoding="utf-8-sig")
#         collected_ids = set(existing_df["post_id"])  # 기존 댓글 ID 추가
#     except FileNotFoundError:
#         print("기존 데이터 없음 (처음 실행)")

# # 댓글 수집
# def collect_comments(driver):
#     global collected_comments
#     WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div._1p19xcx1 article.comment")))

#     comment_elements = driver.find_elements(By.CSS_SELECTOR, "div._1p19xcx1 article.comment")
#     for comment in comment_elements:
#         try:
#             post_id = comment.get_attribute("data-post-anchor-id")
#             if post_id in collected_ids:
#                 continue

#             title_element = comment.find_element(By.CSS_SELECTOR, "div > a > span:nth-child(1) > span")
#             comment_title = title_element.text.strip()

#             text_element = comment.find_element(By.CSS_SELECTOR, "div > a > span:nth-child(2) > span")
#             comment_text = text_element.text.strip()

#             time_element = comment.find_element(By.CSS_SELECTOR, "header > div > label > time")
#             timestamp = time_element.get_attribute('datetime')

#             collected_comments.append((post_id, comment_title, comment_text, timestamp))
#             collected_ids.add(post_id)  # 중복 방지를 위해 ID 저장

#         except Exception as e:
#             print(f"오류 발생: {e}")
#             continue

#     print(f"현재 화면에서 수집한 댓글: {len(collected_comments)}")

# # 댓글 저장
# def save_comments(file_path="test_comments.csv"):
#     if not collected_comments:
#         print("저장할 새 댓글이 없습니다.")
#         return

#     df = pd.DataFrame(collected_comments, columns=["post_id", "title", "comment", "timestamp"])
#     df.to_csv(file_path, mode="a", header=False, index=False, encoding="utf-8-sig")
#     save_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
#     print(f'마지막 저장 시간: {save_time}')
#     collected_comments.clear()  # 저장 후 리스트 초기화

# #스크롤+수집
# def scroll_and_collect(driver, repeat_count=5):
#     """처음 화면에서 댓글을 먼저 수집한 후, 스크롤하면서 추가 댓글을 수집하는 함수"""
    
#     # 1. 첫 화면에서 댓글 수집
#     print("초기 화면 댓글 수집 시작...")
#     collect_comments(driver)  
    
#     for i in range(repeat_count):
#         print(f"{i+1}/{repeat_count} 번째 스크롤 중...")

#         # 2. JavaScript를 이용해 스크롤 다운 (스크롤 속도 최적화)
#         driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

#         # 3. 새로운 댓글이 로딩될 때까지 대기
#         try:
#             WebDriverWait(driver, 5).until(
#                 EC.presence_of_element_located((By.CSS_SELECTOR, "div._1p19xcx1 article.comment"))
#             )
#         except:
#             print("새로운 댓글 로딩 실패, 다음 스크롤 진행")

#         time.sleep(2)  # 페이지가 완전히 로드될 시간을 조금 줌
        
#         # 4. 스크롤 후 댓글 수집
#         collect_comments(driver)

#     print("스크롤 및 댓글 수집 완료.")
        

# #크롤링 전체 실행 함수
# def run_scraper(driver, repeat_count, file_path="test_comments.csv"):
#     load_existing_ids(file_path)  # 기존 데이터 로드
#     scroll_and_collect(driver, repeat_count)  # 스크롤 & 댓글 수집
#     save_comments(file_path)  # 데이터 저장


In [22]:
# 변수선언부
collected_data_index = set()  # 중복 방지를 위한 `data-index` 저장
collected_ids = set()  # 중복 방지용 ID 저장
collected_comments = []  # 수집된 댓글 저장 리스트

# id확인
def load_existing_ids(file_path="test_comments.csv"):
    global collected_ids
    try:
        existing_df = pd.read_csv(file_path, encoding="utf-8-sig")
        collected_ids = set(existing_df["post_id"])  # 기존 댓글 ID 추가
    except FileNotFoundError:
        print("기존 데이터 없음 (처음 실행)")
        
        
def collect_comments(driver):
    """현재 화면에서 `data-index` 기준으로 댓글을 수집하는 함수"""
    global collected_comments, collected_data_index
    
    try:
        # 댓글 요소가 나타날 때까지 대기
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "[data-index]"))
        )

        # 현재 화면에 있는 모든 `data-index` 요소 가져오기
        comment_elements = driver.find_elements(By.CSS_SELECTOR, "[data-index]")

        for comment in comment_elements:
            try:
                # data-index
                data_index = int(comment.get_attribute("data-index"))
                if data_index in collected_data_index:
                    continue

                # post_id
                post_id = comment.get_attribute("data-post-anchor-id")
                if post_id in collected_ids:
                    continue

                # 댓글 contents
                try:
                    title_element = comment.find_element(By.CSS_SELECTOR, "span._1sihfl60")
                    comment_title = title_element.text.strip()
                except:
                    comment_title = ""

                try:
                    text_element = comment.find_element(By.CSS_SELECTOR, "span._60z0ev1._60z0ev6._60z0ev0._1tvp9v41._1sihfl60")
                    comment_text = text_element.text.strip()
                except:
                    comment_text = ""

                time_element = comment.find_element(By.CSS_SELECTOR, "time")
                timestamp = time_element.get_attribute('datetime')

                # 9️⃣ 중복 방지를 위해 리스트 & 셋에 저장
                collected_comments.append((post_id, data_index, comment_title, comment_text, timestamp))
                collected_ids.add(post_id)
                collected_data_index.add(data_index)

            except Exception as e:
                print(f"댓글 수집 중 오류: {e}")
                continue

        print(f"현재 화면에서 {len(collected_comments)}개 댓글 수집 완료! (data-index: {sorted(collected_data_index)})")

    except Exception as e:
        print(f"댓글 수집 중 오류: {e}")
    
    
def scroll_and_collect(driver, repeat_count):
    global collected_data_index
    print("초기 화면 댓글 수집 시작")
    collect_comments(driver)
               
    for i in range(repeat_count):
        # 마지막 수집 data-index 저장
        prev_max_index = max(collected_data_index) if collected_data_index else 0
        print(f"🔽 {i+1}/{repeat_count} 번째 스크롤 진행... (현재 최대 data-index: {prev_max_index})")

        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN)
        time.sleep(2)
        
        WebDriverWait(driver, 10).until(
            lambda d: any(int(el.get_attribute("data-index")) > prev_max_index
                         for el in d.find_elements(By.CSS_SELECTOR, "[data-index]"))
        )
        collect_comments(driver)

    print("스크롤 및 댓글 수집 완료!")


def save_comments(file_path="test_comments.csv"):
    if not collected_comments:
        print("저장할 새 댓글이 없습니다.")
        return

    df = pd.DataFrame(collected_comments, columns=["post_id", "title", "comment", "timestamp"])
    df.to_csv(file_path, mode="a", header=False, index=False, encoding="utf-8-sig")
    save_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f'마지막 저장 시간: {save_time}')
    collected_comments.clear()  # 저장 후 리스트 초기화

In [28]:
driver.quit()

KeyboardInterrupt: 

In [20]:
#크롤링 전체 실행 함수
def run_scraper(driver, repeat_count, file_path="test_comments.csv"):
    load_existing_ids(file_path)  # 기존 데이터 로드
    scroll_and_collect(driver, repeat_count)  # 스크롤 & 댓글 수집
    save_comments(file_path)  # 데이터 저장

In [ ]:
setting()
sorting()
run_scraper(driver, repeat_count=5, file_path="test_comments.csv")

현재 정렬이 최신순입니다.
기존 데이터 없음 (처음 실행)
초기 화면 댓글 수집 시작
댓글 수집 중 오류: Message: no such element: Unable to locate element: {"method":"css selector","selector":"time"}
  (Session info: chrome=134.0.6998.89); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x006DB5A3+24387]
	(No symbol) [0x00665904]
	(No symbol) [0x00540753]
	(No symbol) [0x00588BA9]
	(No symbol) [0x00588EFB]
	(No symbol) [0x0057E241]
	(No symbol) [0x005AD894]
	(No symbol) [0x0057E164]
	(No symbol) [0x005ADAC4]
	(No symbol) [0x005CF138]
	(No symbol) [0x005AD646]
	(No symbol) [0x0057C59F]
	(No symbol) [0x0057D8E4]
	GetHandleVerifier [0x009DD883+3179043]
	GetHandleVerifier [0x009F6CF9+3282585]
	GetHandleVerifier [0x009F167C+3260444]
	GetHandleVerifier [0x00774330+650448]
	(No symbol) [0x0066ED0D]
	(No symbol) [0x0066BAF8]
	(No symbol) [0x0066BC99]
	(No symbol) [0x0065E530]
	BaseThreadInitThunk [0x7634FCC9+2

ValueError: 4 columns passed, passed data had 5 columns

In [31]:
df = pd.read_csv('test_comments.csv')
df

,post_id,data_index,title,comment,timestamp
0,102245622,2,NaN,배당락 끝나고 튀려나?,2025-03-14T18:24:34+09:00
1,102243550,3,6만 언제가누,기다리다 죽겠다,2025-03-14T18:09:28+09:00
2,102240920,4,주차장 입장,맨날 주차해 ㅋㅋ 이것도 능력이다,2025-03-14T17:51:16+09:00
3,102240530,5,‘오징어게임’에 비유된 서학개미의 투자 : 여러 의미를 생각하게 하다.,얼마 전 미국의 자산운용사 아카디안(Acadian)의 라몬트 수석 부사장은 “오징어...,2025-03-14T17:48:51+09:00
4,102230270,6,좋은일 있으리라,좋은 일 있으리라\n크게 있으리라\n많이 있으리라,2025-03-14T16:36:10+09:00
...,...,...,...,...,...
205,102184678,153,NaN,사팔사팔인겁니까 재봉은 겁나 치고있는데 ㅋㅋㅋ,2025-03-14T09:43:37+09:00
206,102184394,154,NaN,익절 하고 싶은데 그냥 손절 할까요? ㅠㅠ,2025-03-14T09:41:34+09:00
207,102184114,155,거래이력 있어야 출금,ㅇ,2025-03-14T09:39:25+09:00
208,102183812,156,NaN,오늘은 올라가는척도 안하네ㅋㅋㅋㅋㅋㅋㅋㅋㅋ,2025-03-14T09:37:23+09:00


In [ ]:
df.isna()



,post_id,data_index,title,comment,timestamp
0,True,False,True,False,False
1,False,False,True,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
59,False,False,False,False,False
60,False,False,False,False,False
61,False,False,True,False,False
62,False,False,True,False,False


In [ ]:
df

In [33]:
import pandas as pd

# CSV 파일을 DataFrame으로 읽기
file_path = 'test_comments.csv'  # 여기에 CSV 파일의 경로를 입력하세요
df = pd.read_csv(file_path)

# 중복된 post_id가 있는 행 출력
duplicate_post_ids = df[df.duplicated(subset='post_id', keep=False)]
print("중복된 post_id:")
print(duplicate_post_ids)

# 중복된 post_id 제거하기
df_unique = df.drop_duplicates(subset='post_id', keep='first')  # 'first'는 첫 번째 항목을 유지

# 중복 제거 후 DataFrame 확인
print("중복 제거 후 DataFrame:")
print(df_unique)

# 중복 제거된 DataFrame을 CSV로 저장할 수도 있습니다.
df_unique.to_csv('unique_comments.csv', index=False)


중복된 post_id:
       post_id  data_index                                    title  \
0    102245622           2                                      NaN   
1    102243550           3                                  6만 언제가누   
2    102240920           4                                   주차장 입장   
3    102240530           5  ‘오징어게임’에 비유된 서학개미의 투자 : 여러 의미를 생각하게 하다.   
4    102230270           6                                 좋은일 있으리라   
..         ...         ...                                      ...   
106  102221952          54                                   세계는 지금   
107  102221002          55                         얘는 뭐 박음질 하고있는거임?   
108  102220402          56                                      NaN   
109  102220272          57                                      NaN   
110  102219970          58                                       추매   

                                               comment  \
0                                         배당락 끝나고 튀려나?   
1                 

In [53]:
stock1 = pd.read_csv('SK이노베이션.csv')
stock2 = pd.read_csv('NAVER.csv')
stock3 = pd.read_csv('삼성전자.csv')

In [54]:
print(len(stock1), len(stock2), len(stock3))

13 14 766


In [49]:
stock1.head()

,platform,stock_name,timestamp,post_id,data_index,title,comment
0,토스증권,SK이노베이션,2025-03-17T14:34:21+09:00,102472784,2,NaN,한달동안 오른게 이틀만에 무너지는구나...
1,토스증권,SK이노베이션,2025-03-17T14:32:41+09:00,102472612,3,NaN,진짜 지옥까지 가는구나....
2,토스증권,SK이노베이션,2025-03-17T13:00:16+09:00,102463270,4,NaN,언제까지 매수해야 배당금 받을 수 있을까요?\n배당락도 감수 해야겠지요?^^
3,토스증권,SK이노베이션,2025-03-17T12:26:24+09:00,102460010,5,추매,"평단 127,900\n주주분들 화이팅하세요~\n웃는날 있을거예요~"
4,토스증권,SK이노베이션,2025-03-17T12:12:27+09:00,102458354,6,NaN,어이가 없네
